In [1]:
from IPython.display import clear_output
import numpy as np
from framework.io.io_functions import Reader, Writer
from framework.base.dataset import Dataset
from framework.reconstruction.parameter import Parameter
from framework.transformers.dfts import DFT1D, NUFFT1D
import matplotlib.pyplot as plt
from framework.objectivefunction.ofunction import OFunction
from framework.utils.utilities import real_to_complex, complex_to_real, find_pixel
from framework.objectivefunction.priors import TSV, TV, L1, Chi2
from framework.optimization.optimizer import FISTA, ADMM, SDMM, GradientBasedMethod
from framework.utils.analytical_functions import Gaussian
from framework.transformers.gridding import Gridding
from scipy.constants import c
from pynufft import NUFFT
from scipy import signal as sci_signal
import itertools

ModuleNotFoundError: No module named 'framework.optimization.parameter'

In [ ]:
# Read data
P = np.load("polarized_emission.npy")
sigma = np.load("sigmas.npy")
nu = np.load("freqs.npy")

In [ ]:
measurements = Dataset(nu=nu, data=P, sigma=sigma)

In [ ]:
plt.plot(measurements.lambda2, measurements.data.real, 'k.', label=r"Stokes $Q$")
plt.plot(measurements.lambda2, measurements.data.imag, 'c.', label=r"Stokes $U$")
#plt.plot(measurements.lambda2, np.abs(measurements.data), 'g.', label=r"$|P|$")
plt.xlabel(r'$\lambda^2$[m$^{2}$]')
plt.ylabel(r'Jy/beam')
plt.legend(loc='upper right')
plt.tight_layout()
plt.savefig('polarized.png', dpi=100, bbox_inches='tight')

In [ ]:
parameter = Parameter()

In [ ]:
parameter.calculate_cellsize(dataset=measurements)

In [ ]:
nufft = NUFFT1D(dataset=measurements, parameter=parameter, solve=True)

In [ ]:
dft = DFT1D(dataset=measurements, parameter=parameter)

In [ ]:
F_dirty = dft.backward(measurements.data)

In [ ]:
plt.plot(parameter.phi, F_dirty.real, 'c--', label=r"Stokes $Q$")
plt.plot(parameter.phi, F_dirty.imag, 'c:', label=r"Stokes $U$")
plt.plot(parameter.phi, np.abs(F_dirty), 'k-', label=r"|P|")
plt.xlabel(r'$\phi$[rad m$^{-2}$]')
plt.ylabel(r'Jy/beam m$^2$ rad$^{-1}$ rmtf$^{-1}$')
plt.legend(loc='upper right')
plt.xlim([-1000,1000])
plt.tight_layout()
plt.savefig('fdirty.png', dpi=100, bbox_inches='tight')

In [ ]:
idx_noise = np.where(np.abs(parameter.phi)>14000)
noise_F = np.std(0.5*(F_dirty[idx_noise].real + F_dirty[idx_noise].imag))
noise_F

In [ ]:
P_back = nufft.forward_normalized(F_dirty)

In [ ]:
plt.plot(measurements.lambda2, P_back.real, 'k.', label=r"Stokes $Q$")
plt.plot(measurements.lambda2, P_back.imag, 'c.', label=r"Stokes $U$")
plt.plot(measurements.lambda2, np.abs(P_back), 'g.', label=r"$|P|$")

In [ ]:
F_back = nufft.backward(measurements.data)

In [ ]:
plt.plot(parameter.phi, F_back.real, 'c--', label=r"Real part")
plt.plot(parameter.phi, F_back.imag, 'c:', label=r"Imaginary part")
plt.plot(parameter.phi, np.abs(F_back), 'k-', label=r"Amplitude")
plt.xlim([-1000,1000])

In [ ]:
P_back = nufft.forward_normalized(F_back)
plt.plot(measurements.lambda2, P_back.real, 'k.', label=r"Stokes $Q$")
plt.plot(measurements.lambda2, P_back.imag, 'c.', label=r"Stokes $U$")
plt.plot(measurements.lambda2, np.abs(P_back), 'g.', label=r"$|P|$")

In [ ]:
lambda_l1 = np.sqrt(2 * len(P) + 4 * np.sqrt(len(P)))*noise_F
#lambda_l1 /= 10
#lambda_l1=0.5
lambda_tsv = 0.0
chi2 = Chi2(dft_obj=nufft)
l1 = L1(reg=lambda_l1)
tsv = TSV(reg=lambda_tsv)
# F_func = [chi2(P, dft, W), L1(lambda_l1)]
F_func = [chi2, l1, tsv]
f_func = [chi2]
g_func = [l1, tsv]

F_obj = OFunction(F_func)
f_obj = OFunction(f_func)
g_obj = OFunction(g_func)

In [ ]:
len(F_back)

In [ ]:
len(parameter.data)

In [ ]:
parameter.data = F_back

In [ ]:
parameter.n

In [ ]:
parameter.complex_data_to_real()

In [ ]:
nufft.parameter.n

In [ ]:
parameter.n

In [ ]:
opt = FISTA(guess_param=parameter, F_obj=F_obj, fx=chi2, gx=g_obj, noise=noise_F, verbose=True)
#opt = FISTA(i_guess=F_real, F_obj=F_obj, fx=chi2, gx=g_obj, maxiter=1000, noise=1e-5, verbose=True)
obj, X = opt.run()

In [ ]:
X.real_data_to_complex()

plt.plot(parameter.phi, X.data.real, 'c--', label=r"Stokes $Q$")
plt.plot(parameter.phi, X.data.imag, 'c:', label=r"Stokes $U$")
plt.plot(parameter.phi, np.abs(X.data), 'k-', label=r"$|P|$")
plt.xlabel(r'$\phi$[rad m$^{-2}$]')
plt.ylabel(r'Jy/beam m$^2$ rad$^{-1}$ pix$^{-1}$')
plt.legend(loc='upper right')
plt.tight_layout()
plt.xlim([-1000,1000])

In [ ]:
plt.plot(measurements.lambda2, measurements.model_data.real, 'k.', label=r"Stokes $Q$")
plt.plot(measurements.lambda2, measurements.model_data.imag, 'c.', label=r"Stokes $U$")
plt.plot(measurements.lambda2, np.abs(measurements.model_data), 'g.', label=r"$|P|$")
#plt.ylim([-0.007,0.008])

In [ ]:
plt.plot(measurements.lambda2, measurements.residual.real, 'k.', label=r"Stokes $Q$")
plt.plot(measurements.lambda2, measurements.residual.imag, 'c.', label=r"Stokes $U$")
plt.plot(measurements.lambda2, np.abs(measurements.residual), 'g.', label=r"$|P|$")

In [ ]:
gridding = Gridding(measurements)

In [ ]:
gridded_data = gridding.run()

In [ ]:
np.mean(measurements.residual.real)

In [ ]:
np.mean(measurements.residual.imag)

In [ ]:
hist_real,bins_real,hist_imag,bins_imag = gridded_data.histogram_residuals()

In [ ]:
plt.bar(bins_real[:-1], hist_real, width=np.diff(bins_real), edgecolor="black", align="edge")

In [ ]:
F_residual = nufft.backward(measurements.residual)
plt.plot(parameter.phi, F_residual.real, 'c--', label=r"Stokes $Q$")
plt.plot(parameter.phi, F_residual.imag, 'c:', label=r"Stokes $U$")
plt.plot(parameter.phi, np.abs(F_residual), 'k-', label=r"$|P|$")
plt.xlim([-1000,1000])

In [ ]:
lags, autocorr_res, bound, percentage_real_in, percentage_imag_in = measurements.assess_residuals()

In [ ]:
plt.plot(lags, autocorr_res.real, 'c-', label=r"Real part")
plt.plot(lags, autocorr_res.imag, 'c--', label=r"Imaginary part")
plt.axhline(-bound)
plt.axhline(bound)
plt.axhline(-bound)
plt.axhline(bound)
plt.grid()
plt.grid()

In [ ]:
percentage_real_in, percentage_imag_in

In [ ]:
conv_model = X.convolve() + F_residual

In [ ]:
plt.plot(parameter.phi, conv_model.real, 'c--', label=r"Stokes $Q$")
plt.plot(parameter.phi, conv_model.imag, 'c:', label=r"Stokes $U$")
plt.plot(parameter.phi, np.abs(conv_model), 'k-', label=r"$|P|$")
plt.xlim([-1000,1000])
plt.xlabel(r'$\phi$[rad m$^{-2}$]')
plt.ylabel(r'Jy/beam m$^2$ rad$^{-1}$ rmtf$^{-1}$')
plt.legend(loc='upper right')
plt.tight_layout()
plt.savefig("restored.png", dpi=100)